In [6]:
import os
import numpy as np
import torch
from sklearn.preprocessing import scale
from torch.utils.data import Dataset, DataLoader
import shutil 
import math


In [9]:
time_step = 16

In [7]:
root_dir = "../../data"

In [21]:
data = np.load(os.path.join(root_dir, "music_train_x.npy"))
label = np.load(os.path.join(root_dir, "music_train_y.npy"))
print(data.shape)
print(label.shape)

(245250, 1024, 2)
(245250, 128)


In [10]:
feature_dim = data.shape[-2] // time_step

In [11]:
r = data[:,:,0]

In [12]:
r = r.reshape(r.shape[0], time_step, feature_dim)

In [15]:
num_classes = label.shape[-1]
num_classes

128

In [22]:
label_new = np.argmax(label, axis=1)

In [29]:
label[100]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [30]:
label_new[100]

48

IndexError: invalid index to scalar variable.

NameError: name 'b' is not defined

(1.0, 1.5707963267948966)

In [13]:
data

array([[[[-6.94761317e-03, -5.00087017e-02],
         [-1.02145847e-02, -4.92590674e-02],
         [-1.29380905e-02, -4.67089395e-02],
         ...,
         [ 3.68595717e-02,  2.63316683e-01],
         [ 1.40977554e-01,  2.32239017e-01],
         [ 2.31159793e-01,  1.70037870e-01]],

        [[-4.07877859e-01, -2.70551381e-01],
         [-4.69729709e-01,  1.68792543e-01],
         [-1.39194888e-01,  4.67156264e-01],
         ...,
         [ 5.76811492e-02,  3.72623263e-01],
         [ 2.90586409e-01,  1.50457347e-01],
         [ 2.31532611e-01, -1.23711850e-01]],

        [[ 1.51814685e-01, -1.11622818e-01],
         [ 1.56215509e-01, -1.10753147e-01],
         [ 1.60630515e-01, -1.09811216e-01],
         ...,
         [-8.94240242e-02, -6.34347700e-02],
         [-9.07070665e-02, -6.39418784e-02],
         [-9.20584328e-02, -6.45844608e-02]],

        ...,

        [[-7.51024785e-02,  1.82505362e-02],
         [-1.04344181e-01,  2.50806316e-02],
         [-1.32248155e-01,  3.83975593

In [22]:
data1 = data.transpose(0, 1, 3, 2)
data1

array([[[[-6.94761317e-03, -1.02145847e-02, -1.29380905e-02, ...,
           3.68595717e-02,  1.40977554e-01,  2.31159793e-01],
         [-5.00087017e-02, -4.92590674e-02, -4.67089395e-02, ...,
           2.63316683e-01,  2.32239017e-01,  1.70037870e-01]],

        [[-4.07877859e-01, -4.69729709e-01, -1.39194888e-01, ...,
           5.76811492e-02,  2.90586409e-01,  2.31532611e-01],
         [-2.70551381e-01,  1.68792543e-01,  4.67156264e-01, ...,
           3.72623263e-01,  1.50457347e-01, -1.23711850e-01]],

        [[ 1.51814685e-01,  1.56215509e-01,  1.60630515e-01, ...,
          -8.94240242e-02, -9.07070665e-02, -9.20584328e-02],
         [-1.11622818e-01, -1.10753147e-01, -1.09811216e-01, ...,
          -6.34347700e-02, -6.39418784e-02, -6.45844608e-02]],

        ...,

        [[-7.51024785e-02, -1.04344181e-01, -1.32248155e-01, ...,
          -3.04771443e-01, -2.62068588e-01, -2.24777408e-01],
         [ 1.82505362e-02,  2.50806316e-02,  3.83975593e-02, ...,
           3.99791

In [23]:
data1 = data1.reshape(data1.shape[0] * data1.shape[1], data1.shape[2] * data1.shape[3])

array([[-0.00694761, -0.01021458, -0.01293809, ...,  0.26331668,
         0.23223902,  0.17003787],
       [-0.40787786, -0.46972971, -0.13919489, ...,  0.37262326,
         0.15045735, -0.12371185],
       [ 0.15181468,  0.15621551,  0.16063051, ..., -0.06343477,
        -0.06394188, -0.06458446],
       ...,
       [-0.20759182, -0.15384529, -0.10030015, ..., -0.13037296,
        -0.20214288, -0.23993298],
       [-0.05676183, -0.05086403, -0.04434363, ...,  0.0528048 ,
        -0.07458325, -0.19553563],
       [-0.57249077, -0.53091036, -0.44983612, ..., -0.77007387,
        -0.63568325, -0.46482703]])

In [12]:
data1.s

(5589, 32, 2, 1600)

In [56]:
data1[:][:][:][0].shape

(32, 2, 1600)

In [23]:
a

array([[-0.00694761, -0.0500087 ],
       [-0.01021458, -0.04925907],
       [-0.01293809, -0.04670894],
       ...,
       [ 0.03685957,  0.26331668],
       [ 0.14097755,  0.23223902],
       [ 0.23115979,  0.17003787]])

In [21]:
b

array([-0.01021458, -0.04925907])